# Práctica 1: Web Scraping

Conexión a la web de conteo de Coronavirus detallada por país:
https://www.worldometers.info/coronavirus/

Se utilizará Beautiful Soup para hacer Web Scraping, buscando la tabla de datos.
Pandas para pasar la información a un dataframe y ajustar la información para poder enviarla a un fichero CSV o bien enviarla a una base de datos.

## Sección 1: Web Scrapping

En esta sección del código se busca la tabla del origen y se guarda en una tabla.

In [1]:
#importamos las librerías de BeautifulSoup para obtener el código de html y ayudarnos al scraping
#se imprta de librería de request para "conectarse" a la url

from bs4 import BeautifulSoup as soup
from requests import get

#el dataset se encuentra en este página, la cual es actualizada cada minuto
my_url = "https://www.worldometers.info/coronavirus/"

pageHTML = get(my_url)

In [2]:
#se obtiene el código html
soup = soup(pageHTML.text)

#se muestran los primeros 1000 caracteres
soup.text[:1000]

'\n\n\n\n\n\nCoronavirus Update (Live): 1,722,850 Cases and 104,775 Deaths from COVID-19 Virus Pandemic - Worldometer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#example2\').dataTable( {\n        "scrollCollapse": true,\n\t\t"sDom": \'<"bottom"flp><"clear">\',\n        "paging":         false\n    } );\n} );\n\t\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#table3\').dataTable( {\n        "scrollCollapse": true,\n\t\t\t\t "order": [[ 1, \'desc\' ]],\n\t\t"sDom": \'<"bottom"flp><"clear">\',\n        "paging":         false\n    } );\n} );\n\t\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#example\').dataTable( {\n        "scrollCollapse": true,\n\t\t"searching": false,\n\n\t\t"sDom": \'<"top">rt<"bottom"flp><

In [3]:
# con este código obtenemos la tabla que contiene la información de detalle de país que es la que queremos obtener.
# previo a escribir este código, se hizo una búsqueda manual en la página mirando el código de html para ver
# en dónde se encontraba la información, esta revisión se hizo con el browser, utilizando las herramientas de desarrollador
# que permiten ubicar el código señalando el objeto de interés, en este caso la tabla. Para esto utilicé Chrome con las
# herramientas de desarrollador y encontré que el objeto 'table' con la etiqueta 'tbody' contenía los valores
data = []
table = soup.find('table')
table_body = table.find('tbody')

# se van ubicando los saltos de línea para poder ir uno a uno llenando las líneas de la variable de tabla en donde 
# se guardarán los datos
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# se muestran los valores de la tabla para ver cómo viene la información.    
print (data)

[['North America', '537,570', '+729', '19,876', '+54', '34,462', '483,232', '11,906', '', '', '', '', 'North America'], ['Europe', '837,795', '+17,686', '71,870', '+1,807', '202,038', '563,887', '30,260', '', '', '', '', 'Europe'], ['Asia', '280,176', '+5,135', '10,435', '+211', '141,479', '128,262', '6,653', '', '', '', '', 'Asia'], ['South America', '45,165', '+189', '1,817', '+8', '4,692', '38,656', '896', '', '', '', '', 'South America'], ['Oceania', '7,702', '+94', '60', '+4', '3,688', '3,954', '85', '', '', '', '', 'Australia/Oceania'], ['Africa', '13,721', '+182', '704', '+7', '2,314', '10,703', '159', '', '', '', '', 'Africa'], ['', '721', '', '13', '', '619', '89', '10', '', '', '', '', ''], ['World', '1,722,850', '+24,015', '104,775', '+2,091', '389,292', '1,228,783', '49,969', '221', '13.4', '', '', 'All'], ['USA', '503,177', '+301', '18,761', '+14', '27,314', '457,102', '10,917', '1,520', '57', '2,538,888', '7,670', 'North America'], ['Spain', '161,852', '+3,579', '16,353',

In [4]:
# eliminamos los valores que no tienen datos, si es que hay alguno
missingData = []
fixedData = [[]]
for row in data:
    for index,col in enumerate(row):
        if(col == ''):
            row[index] = 0
        
    missingData.append(row)

In [5]:
# definimos las cabeceras de acuerdo a la tabla de la web, colocando el nombre que queramos para las columnas.
# debemos tener cuidado porque la página fuente es actualizada constantemente, desde que inicié la práctica, se han hecho
# 5 cambios y se añaden líneas o columnas y si las columnas no coinciden, esta parte del código debe ser revisada y ajustada
# ya que de otra forma enviará un error, la forma de evitar el error es revisar el nombre de las columnas en la página fuente
# y dejar igual las columnas en un sitio y otro, si hay columnas o líneas no útiles, más adeoante se pueden eliminar.

headers = ["Country", "Total Cases", "New Cases", "Total Deaths","New Deaths", "Total Recovered", "Active Cases", "Serious, Critical", "Tot Cases/1M pop", "Tot Deaths/1M pop", "Total Tests", "Tests/1M pop", "Region"]

# mostramos las cabeceras para ver que todo es correcto
headers

# hasta este punto se ha hecho el scraping y la información está lista para pasar a un dataframe

['Country',
 'Total Cases',
 'New Cases',
 'Total Deaths',
 'New Deaths',
 'Total Recovered',
 'Active Cases',
 'Serious, Critical',
 'Tot Cases/1M pop',
 'Tot Deaths/1M pop',
 'Total Tests',
 'Tests/1M pop',
 'Region']

## Sección 2: Envío de datos a data frame

En esta sección se hace limpieza de datos y se escribe la tabla extraida a un dataframe que permite hacer tratamiento y operaciones sobre los datos.

In [6]:
# importamos pandas y numpy para crear dataframes a partir de la tabla creada y poder manipular mejor la información
import pandas as pd
import numpy as np

# se envía la tabla al dataframe
df = pd.DataFrame()
df = df.append(data)
df.columns = headers

# se muestra la información en formato dataframe
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop,Tot Deaths/1M pop,Total Tests,Tests/1M pop,Region
0,North America,"537,570",+729,"19,876",+54,"34,462","483,232","11,906",0,0,0,0,North America
1,Europe,"837,795","+17,686","71,870","+1,807","202,038","563,887","30,260",0,0,0,0,Europe
2,Asia,"280,176","+5,135","10,435",+211,"141,479","128,262","6,653",0,0,0,0,Asia
3,South America,"45,165",+189,"1,817",+8,"4,692","38,656",896,0,0,0,0,South America
4,Oceania,"7,702",+94,60,+4,"3,688","3,954",85,0,0,0,0,Australia/Oceania
5,Africa,"13,721",+182,704,+7,"2,314","10,703",159,0,0,0,0,Africa
6,0,721,0,13,0,619,89,10,0,0,0,0,0
7,World,"1,722,850","+24,015","104,775","+2,091","389,292","1,228,783","49,969",221,13.4,0,0,All
8,USA,"503,177",+301,"18,761",+14,"27,314","457,102","10,917","1,520",57,"2,538,888","7,670",North America
9,Spain,"161,852","+3,579","16,353",+272,"59,109","86,390","7,371","3,462",350,"355,000","7,593",Europe


In [7]:
## Elimina los signos que impiden convertir a número

df['Total Cases'] = df['Total Cases'].str.replace(',', '')
df['New Cases'] = df['New Cases'].str.replace('+', '')
df['New Cases'] = df['New Cases'].str.replace(',', '')
df['Total Deaths'] = df['Total Deaths'].str.replace(',', '')
df['New Deaths'] = df['New Deaths'].str.replace('+', '')
df['New Deaths'] = df['New Deaths'].str.replace(',', '')
df['Total Recovered'] = df['Total Recovered'].str.replace(',', '')
df['Active Cases'] = df['Active Cases'].str.replace(',', '')
df['Serious, Critical'] = df['Serious, Critical'].str.replace(',', '')
df['Tot Cases/1M pop'] = df['Tot Cases/1M pop'].str.replace(',', '')
df['Tot Deaths/1M pop'] = df['Tot Deaths/1M pop'].str.replace(',', '')

## Elimina los NaN para valores nulos y los sustituye por ceros
df.fillna(0, inplace=True)

In [8]:
# muestra los tipos de dato que de momento todo son object
df.dtypes

Country              object
Total Cases          object
New Cases            object
Total Deaths         object
New Deaths           object
Total Recovered      object
Active Cases         object
Serious, Critical    object
Tot Cases/1M pop     object
Tot Deaths/1M pop    object
Total Tests          object
Tests/1M pop         object
Region               object
dtype: object

In [9]:
# cambia los tipos de dato del dataframe, para poder enviar el dataframe eventualmente a una base de datos.
# en caso de enviarlo a un csv, este paso no tiene tanto impacto, sin embargo si queremos hacer operaciones con el
# dataframe, lo mejor es tener las columas declaradas en el formato correcto:

df = df.astype({"Total Cases": int})
df = df.astype({"New Cases": int})
df = df.astype({"Total Deaths": int})
df = df.astype({"New Deaths": int})
df = df.astype({"Total Recovered": int})
df = df.astype({"Active Cases": int})
df = df.astype({"Serious, Critical": int})
df = df.astype({"Tot Cases/1M pop": float})
df = df.astype({"Tot Deaths/1M pop": float})

# vemos que se han convertido correctamente
df.dtypes

Country               object
Total Cases            int32
New Cases              int32
Total Deaths           int32
New Deaths             int32
Total Recovered        int32
Active Cases           int32
Serious, Critical      int32
Tot Cases/1M pop     float64
Tot Deaths/1M pop    float64
Total Tests           object
Tests/1M pop          object
Region                object
dtype: object

In [10]:
# Removemos columnas y líneas no utilizadas
# es importante señalar que para este caso, he decidido quitar estas columnas, porque el scraping del covid está
# teniendo mucho movimiento porque al tratarse de un tema de interés mundial, la página fuente altera la tabla constantemente
# para aportar más información y en nuestra párctica no toda la información es relavante

# elimino 3 columnas que no estaban en el código inicial.
# en casos futuros, si 

# quita las columnas no utilizadas.
del df['Total Tests']
del df['Tests/1M pop']
del df['Region']

# y quita las líneas no utilizadas que corresponden a agrupaciones y sumas que se pueden hacer por otros medios.
# si las dejamos enviará valores duplicados al hacer sumas.

i = 0
while i < 8:
  df.drop(df.index[0],inplace=True)
  i += 1

# mostramos y validamos la tabla en formato final
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop,Tot Deaths/1M pop
8,USA,503177,301,18761,14,27314,457102,10917,1520.00,57.0
9,Spain,161852,3579,16353,272,59109,86390,7371,3462.00,350.0
10,Italy,147577,0,18849,0,30455,98273,3497,2441.00,312.0
11,France,124869,0,13197,0,24932,86740,7004,1913.00,202.0
12,Germany,122855,684,2736,0,53913,66206,4895,1466.00,33.0
13,UK,78991,5233,9875,917,344,68772,1559,1164.00,145.0
14,Iran,70029,1837,4357,125,41947,23725,3987,834.00,52.0
15,Turkey,47029,0,1006,0,2423,43600,1667,558.00,12.0
16,Belgium,28018,1351,3346,327,5986,18686,1262,2418.00,289.0
17,Switzerland,24900,349,1003,1,11100,12797,386,2877.00,116.0


## Sección 3: Comprobaciones y envío a fichero

En esta sección se comprueba que los valores son correctos vs la fuente y se envían los datos a un fichero CSV para su posterior uso, en otras herramientas de análisis, visualización o predicción.

In [11]:
## Pone el resumen de datos para validar que efectivamente es correcto con respecto a la página fuente

Total_Cases= df['Total Cases'].sum()
Total_Deaths= df['Total Deaths'].sum()
Total_Recovered = df['Total Recovered'].sum()


print ("Total Cases=",Total_Cases)
print ("Total Deaths=",Total_Deaths)
print ("Total Recovered=",Total_Recovered)

Total Cases= 1722850
Total Deaths= 104775
Total Recovered= 389292


In [12]:
## Exporta a fichero .CSV

#reemplazar <path> por el directorio al cual se enviará la lista

df.to_csv('<path>/countries.csv')